# Imports

In [1]:
from functools import wraps

import numpy as np
import qcodes as qc

from labcore.measurement import *

from configuration import QMConfig
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager

from plottr.data import datadict_storage as dds, datadict as dd

# global module variable for the config file
global_config = None


# variable used to change the name of the saved DDH5
test_name_counter = 0


# Base Decorator

In [2]:
class BGRecAsClassBase:
    """
    Base class decorator used to record asynchronous data from instrument. 
    Each instrument should have its own custom setup_wrapper (see setup_wrapper docstring for more info), and a custom collector 
    """
    def __init__(self, *specs):
        
        self.specs = specs  
        self.communicator = {}

        
    def __call__(self, fun):
        def sweep(**collector_kwargs):
            setup_sweep = once(self.setup_wrapper(fun))
            gather_sweep = Sweep(record_as(self.collector(**collector_kwargs), *self.specs))
            return setup_sweep + gather_sweep
        return sweep
    
    
        
    def setup_wrapper(self, fun):
        """
        Wraps the setup function. setup_wrapper should consist of a function wrapped by the decorator @wraps and takes fun as an arugment.
        In this case the wrapped function is setup. 
        Setup should accpet the *args and **kwargs of fun. It should also place any returns from fun in the communicator.
        setup_wrapper needs to return the wraped function (setup)
        """
        @wraps(fun)
        def setup(*args, **kwargs):
            self.communicator['setup_return'] = fun(*args, **kwargs)
            return None
        
        return setup
    
    def collector(self):
        """
        Data gathering function. This function should be a generator that collects data from the instrument.
        All the logic of asynchronous data collection should be here. It should yield data as long as it is 
        available and the generator should finish once its experiment is done.
        """
        yield None
    
    
def create_background_sweep(decorated_setup_function, **collector_kwargs):
    sweep = decorated_setup_function(**collector_kwargs)
    return sweep


# Specific OPX Implementation

In [3]:
class RecordOPX(BGRecAsClassBase):
    def setup_wrapper(self, fun):
        @wraps(fun)
        def setup(*args, **kwargs):
            qmachine_mgr = QuantumMachinesManager()
            qmachine = qmachine_mgr.open_qm(global_config)
            job = qmachine.execute(fun(*args, **kwargs))
            result_handles = job.result_handles
            self.communicator['result_handles'] = result_handles
            self.communicator['active'] = True
            self.communicator['counter'] = 0
            
        return setup
    
    
    def _control_progress(self, batchsize):
        ready = False
        first = True
        res_handle = self.communicator['result_handles']
        counter = self.communicator['counter']
        while not ready:
            for name, handle in res_handle:
                    current = handle.count_so_far()
                    
                    if res_handle.is_processing():
                        if current - counter >= batchsize:
                            ready = True
                        else:
                            ready = False
                    else:
                        ready = True
                        self.communicator['active'] = False

                        
        return ready
    

    def collector(self, batchsize):


        result_handle = self.communicator['result_handles']
        
        # checking if the parameters passed match the parameters in the opx
        data_specs_names = [x.name for x in self.specs]
        variable_counter = 0
        for name, handle in result_handle:

            if name not in data_specs_names:
                raise ValueError(f'{name} is not a recorded variable')
            else:
                variable_counter += 1
        if variable_counter != len(data_specs_names):
            raise ValueError(f'Number of recorded variables ({variable_counter}) \
                             does not match number of variables gathered from the OPX ({len(data_specs_names)})')
        
        
        while self.communicator['active']:

            first = True
            data = {}
            counter = self.communicator['counter']
            first = True
            current = 0
            
            if result_handle == None:
                yield None
            
            record = self._control_progress(batchsize)  
            for name, handle in result_handle:

                if first:
                    current = handle.count_so_far()

                    first = False
                    if current == counter:
                        yield None
                    
                    
                handle.wait_for_values(current)
                data_temp = np.array(handle.fetch(slice(counter, current)))     
                if name[0:4] == 'raw_':
                    holding_converting = []
                    for i in data_temp:
                        i_holder = []
                        for j in i:
                            converted = j.astype(float)
                            i_holder.append(converted)
                        holding_converting.append(i_holder)
                    if len(holding_converting) == 1:
                        converted_data_temp = [np.squeeze(holding_converting)]
                    else:
                        converted_data_temp = np.squeeze(holding_converting)
                else:
                    converted_data_temp = data_temp.astype(float)
                data[name] = converted_data_temp
            self.communicator['counter'] = current
            yield data
            

# Proposal for Base Running and Saving

In [4]:
def _create_structure(sweep):
    data_specs = sweep.get_data_specs()
    data_dict = dd.DataDict()
    for spec in data_specs:

        depends_on = spec.depends_on
        unit = spec.unit
        name = spec.name
        if depends_on is None:
            if unit is None:
                data_dict[name] = dict()
            else:
                data_dict[name] = dict(unit = unit)
        else:
            if unit is None:
                data_dict[name] = dict(axes = depends_on)
            else:
                data_dict[name] = dict(axes = depends_on, unit = unit)
                    
    data_dict.validate()

    return data_dict

def _check_none(argument):
    for arg in argument.keys():
        if argument[arg] is not None:
            return False
    return True

def run_and_save_sweep(sweep, data_dir, name, prt=False):
    data_dict = _create_structure(sweep)
    with dds.DDH5Writer(data_dir, data_dict, name=name) as writer:
        for line in sweep:
            if not _check_none(line):
                if prt:
                    print(line)
                try:
                    writer.add_data(**line)
                except ValueError as e:
                    raise e
                    
    print('The measurement has finished and all of the data has been saved.')

# QUA experiment with implemented decorator

In [23]:
@RecordOPX(
        independent('repetition', type='array'),
        dependent('V', depends_on=['repetition'], type='array'),
        dependent('tracker', depends_on=['repetition'], type='array'),
        dependent('raw_values', depends_on=['repetition'], type='array'))
def my_qua_experiment(n_reps=1000):
    with program() as qua_measurement:
        raw_stream = declare_stream(adc_trace=True)
        v_stream = declare_stream()
        tracker_stream = declare_stream()
        i_stream = declare_stream()

        i = declare(int)
        v = declare(fixed)
        tracker = declare(int, value=0)

        with for_(i, 0, i<n_reps, i+1):
            save(i, i_stream)
            
            measure('box', 'readout', raw_stream, ("box_sin",v))
            save(v, v_stream)

            assign(tracker, tracker+2)
            save(tracker, tracker_stream)

            play('box', "readout")
            wait(1000000)

        with stream_processing():
            i_stream.save_all('repetition')
            v_stream.save_all('V')
            tracker_stream.save_all('tracker')
            raw_stream.input1().timestamps().save_all('raw_values')

    return qua_measurement

In [24]:
DATADIR = './data/'
config = QMConfig()
global_config = config.config()

sweep = create_background_sweep(my_qua_experiment, batchsize=5)

In [25]:
sweep.set_action_opts(
    my_qua_experiment=dict(n_reps=25)
)

In [26]:
run_and_save_sweep(sweep, DATADIR, f'OPX test standard #{test_name_counter}', prt=True)

Data location:  ./data/2021-07-28\2021-07-28_0017_OPX test standard #0\2021-07-28_0017_OPX test standard #0.ddh5
2021-07-28 18:28:53,553 - qm - INFO - Performing health check
2021-07-28 18:28:53,554 - qm - INFO - Health check passed
2021-07-28 18:28:53,855 - qm - INFO - Flags: 
2021-07-28 18:28:53,856 - qm - INFO - Executing high level program
{'repetition': array([0., 1., 2., 3., 4., 5.]), 'V': array([-0.19574275, -0.19574419, -0.19577436, -0.19576854, -0.19578452,
       -0.19573664]), 'tracker': array([ 2.,  4.,  6.,  8., 10., 12.]), 'raw_values': array([[4.0000000e+02, 4.0100000e+02, 4.0200000e+02, ..., 1.0397000e+04,
        1.0398000e+04, 1.0399000e+04],
       [4.0204000e+06, 4.0204010e+06, 4.0204020e+06, ..., 4.0303970e+06,
        4.0303980e+06, 4.0303990e+06],
       [8.0404000e+06, 8.0404010e+06, 8.0404020e+06, ..., 8.0503970e+06,
        8.0503980e+06, 8.0503990e+06],
       [1.2060400e+07, 1.2060401e+07, 1.2060402e+07, ..., 1.2070397e+07,
        1.2070398e+07, 1.2070399e+

# Notes

* Right now, to indicate that a variable is saving the raw trace, the variable needs to start with 'raw_'.
* The run_and_save function should work with any sweep, so that could be added to the general labcore package.
* If a measurement fails to be executed, the writter still creates a file but the file is corrupted and plottr jsut raises a warning.
* I'll start working on properly documenting the code tomorrow

